In [3]:
import pandas as pd
import numpy as np
from epysurv.models.timepoint import FarringtonFlexible
import matplotlib.pyplot as plt
import os
import requests
from sodapy import Socrata

c:\Users\taohe\miniconda3\envs\epysurv-dev\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [5]:


def getCdcData(where=None, select=None):
    try:
        app_token="Wa9PucgUy1cHNJgzoTZwhg9AY"
        client = Socrata("data.cdc.gov", app_token=app_token, timeout=120)
        
        all_results = []
        offset = 0
        while True:  # Adjust the limit as needed
            results = client.get("r8kw-7aab", limit=5000, offset=offset, where=where, select=select)
            if not results:  # Break if no more data is returned
                break
            all_results.extend(results)
            offset += 5000  # Increment the offset for the next chunk

        results_df = pd.DataFrame.from_records(all_results)
        return results_df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    


In [48]:
df = getCdcData( select="start_date, end_date,  mmwr_week, state, covid_19_deaths, total_deaths, pneumonia_deaths, pneumonia_and_covid_19_deaths, influenza_deaths,pneumonia_influenza_or_covid_19_deaths")


In [49]:
print(df)

                    start_date                 end_date mmwr_week  \
0      2019-12-29T00:00:00.000  2020-01-04T00:00:00.000         1   
1      2020-01-05T00:00:00.000  2020-01-11T00:00:00.000         2   
2      2020-01-12T00:00:00.000  2020-01-18T00:00:00.000         3   
3      2020-01-19T00:00:00.000  2020-01-25T00:00:00.000         4   
4      2020-01-26T00:00:00.000  2020-02-01T00:00:00.000         5   
...                        ...                      ...       ...   
19381  2020-01-01T00:00:00.000  2025-06-21T00:00:00.000       NaN   
19382  2020-01-01T00:00:00.000  2025-06-21T00:00:00.000       NaN   
19383  2020-01-01T00:00:00.000  2025-06-21T00:00:00.000       NaN   
19384  2020-01-01T00:00:00.000  2025-06-21T00:00:00.000       NaN   
19385  2020-01-01T00:00:00.000  2025-06-21T00:00:00.000       NaN   

               state covid_19_deaths total_deaths pneumonia_deaths  \
0      United States               0        60170             4111   
1      United States           

In [50]:
df_week=df[df['mmwr_week']>='1']

In [51]:
df_week_us=df_week[df_week['state']=='United States']

In [52]:
print(df_week_us)

                  start_date                 end_date mmwr_week  \
0    2019-12-29T00:00:00.000  2020-01-04T00:00:00.000         1   
1    2020-01-05T00:00:00.000  2020-01-11T00:00:00.000         2   
2    2020-01-12T00:00:00.000  2020-01-18T00:00:00.000         3   
3    2020-01-19T00:00:00.000  2020-01-25T00:00:00.000         4   
4    2020-01-26T00:00:00.000  2020-02-01T00:00:00.000         5   
..                       ...                      ...       ...   
281  2025-05-18T00:00:00.000  2025-05-24T00:00:00.000        21   
282  2025-05-25T00:00:00.000  2025-05-31T00:00:00.000        22   
283  2025-06-01T00:00:00.000  2025-06-07T00:00:00.000        23   
284  2025-06-08T00:00:00.000  2025-06-14T00:00:00.000        24   
285  2025-06-15T00:00:00.000  2025-06-21T00:00:00.000        25   

             state covid_19_deaths total_deaths pneumonia_deaths  \
0    United States               0        60170             4111   
1    United States               1        60734            

In [53]:
df=df_week_us[['start_date', 'end_date', 'mmwr_week',  'pneumonia_deaths' ]]

In [54]:
print(df)

                  start_date                 end_date mmwr_week  \
0    2019-12-29T00:00:00.000  2020-01-04T00:00:00.000         1   
1    2020-01-05T00:00:00.000  2020-01-11T00:00:00.000         2   
2    2020-01-12T00:00:00.000  2020-01-18T00:00:00.000         3   
3    2020-01-19T00:00:00.000  2020-01-25T00:00:00.000         4   
4    2020-01-26T00:00:00.000  2020-02-01T00:00:00.000         5   
..                       ...                      ...       ...   
281  2025-05-18T00:00:00.000  2025-05-24T00:00:00.000        21   
282  2025-05-25T00:00:00.000  2025-05-31T00:00:00.000        22   
283  2025-06-01T00:00:00.000  2025-06-07T00:00:00.000        23   
284  2025-06-08T00:00:00.000  2025-06-14T00:00:00.000        24   
285  2025-06-15T00:00:00.000  2025-06-21T00:00:00.000        25   

    pneumonia_deaths  
0               4111  
1               4153  
2               4066  
3               3915  
4               3818  
..               ...  
281             2916  
282        

In [55]:

# 转换为 datetime 类型
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])

c:\Users\taohe\miniconda3\envs\epysurv-dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\taohe\miniconda3\envs\epysurv-dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
# 设置 index 为每周开始时间（推荐）
df = df.set_index("start_date")
df = df.sort_index()

In [82]:
print(df.index)

DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2025-04-13', '2025-04-20', '2025-04-27', '2025-05-04',
               '2025-05-11', '2025-05-18', '2025-05-25', '2025-06-01',
               '2025-06-08', '2025-06-15'],
              dtype='datetime64[ns]', name='start_date', length=286, freq=None)


In [77]:
df = df.rename(columns={"pneumonia_deaths": "n_cases"})

In [78]:
df["n_cases"] = df["n_cases"].astype(int)

In [79]:
df['n_outbreak_cases'] = df['n_cases'].apply(lambda x: 0 if x <= 1000 else x - 1000)

In [83]:
df.index = pd.date_range(start=df.index[0], periods=len(df), freq='W-SUN')

In [84]:
print(df.index)

DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2025-04-13', '2025-04-20', '2025-04-27', '2025-05-04',
               '2025-05-11', '2025-05-18', '2025-05-25', '2025-06-01',
               '2025-06-08', '2025-06-15'],
              dtype='datetime64[ns]', length=286, freq='W-SUN')


In [85]:
train_end_date= "2024-12-30" 
test_end_date="2025-01-06"

In [127]:
# 用 train/test date 训练
train = df.loc[:train_end_date, ['n_cases', 'n_outbreak_cases']].copy()
test = df.loc[train_end_date:, ['n_cases', 'n_outbreak_cases']].copy()



In [128]:
expected_index = pd.date_range(start=train.index[0], end=train.index[-1], freq='W-SUN')
missing = expected_index.difference(train.index)
print("Missing weeks in train:", missing)

Missing weeks in train: DatetimeIndex([], dtype='datetime64[ns]', freq=None)


In [103]:
expected_index = pd.date_range(start=test.index[0], end=test.index[-1], freq='W-SUN')
missing = expected_index.difference(test.index)
print("Missing weeks in test:", missing)

Missing weeks in test: DatetimeIndex([], dtype='datetime64[ns]', freq=None)


In [129]:
print(expected_index)

DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2024-10-27', '2024-11-03', '2024-11-10', '2024-11-17',
               '2024-11-24', '2024-12-01', '2024-12-08', '2024-12-15',
               '2024-12-22', '2024-12-29'],
              dtype='datetime64[ns]', length=262, freq='W-SUN')


In [130]:
model = FarringtonFlexible(alpha=0.05, years_back=3)
model.fit(train)


FarringtonFlexible(years_back=3, window_half_width=3, reweight=True, weights_threshold=2.58, alpha=0.05, trend=True, trend_threshold=0.05, past_period_cutoff=4, min_cases_in_past_periods=5, power_transform='2/3', past_weeks_not_included=26, threshold_method='delta')

In [131]:
print(train.index)

DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2024-10-27', '2024-11-03', '2024-11-10', '2024-11-17',
               '2024-11-24', '2024-12-01', '2024-12-08', '2024-12-15',
               '2024-12-22', '2024-12-29'],
              dtype='datetime64[ns]', length=262, freq='W-SUN')


In [132]:
print("Inferred frequency:", pd.infer_freq(test.index))

Inferred frequency: W-SUN


In [133]:
full_data = (
            pd.concat((train, test), keys=["train", "test"], sort=True)
            .reset_index(level=0)
            .rename(columns={"level_0": "provenance"})
        )

In [134]:
d1=pd.concat((train, test), keys=["train", "test"], sort=True)

In [119]:
d2=d1.reset_index(level=0)

In [136]:
print(full_data.index)

DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2025-04-13', '2025-04-20', '2025-04-27', '2025-05-04',
               '2025-05-11', '2025-05-18', '2025-05-25', '2025-06-01',
               '2025-06-08', '2025-06-15'],
              dtype='datetime64[ns]', length=286, freq='W-SUN')


In [137]:
if full_data.index.freq is None:
    freq = pd.infer_freq(full_data.index)
    print("Inferred frequency:", freq)


In [121]:
# Step 1: Add provenance column manually before concat
train_ = train.copy()
train_["provenance"] = "train"

test_ = test.copy()
test_["provenance"] = "test"

# Step 2: Concatenate directly along axis=0 (row-wise), keep original index
full_data = pd.concat([train_, test_], axis=0).sort_index()

In [126]:
print(test_.index)

DatetimeIndex(['2025-01-12', '2025-01-19', '2025-01-26', '2025-02-02',
               '2025-02-09', '2025-02-16', '2025-02-23', '2025-03-02',
               '2025-03-09', '2025-03-16', '2025-03-23', '2025-03-30',
               '2025-04-06', '2025-04-13', '2025-04-20', '2025-04-27',
               '2025-05-04', '2025-05-11', '2025-05-18', '2025-05-25',
               '2025-06-01', '2025-06-08', '2025-06-15'],
              dtype='datetime64[ns]', freq='W-SUN')


In [107]:
predictions = model.predict(test)

ValueError: The time series index has no valid frequency. Index=DatetimeIndex(['2019-12-29', '2020-01-05', '2020-01-12', '2020-01-19',
               '2020-01-26', '2020-02-02', '2020-02-09', '2020-02-16',
               '2020-02-23', '2020-03-01',
               ...
               '2025-04-13', '2025-04-20', '2025-04-27', '2025-05-04',
               '2025-05-11', '2025-05-18', '2025-05-25', '2025-06-01',
               '2025-06-08', '2025-06-15'],
              dtype='datetime64[ns]', length=285, freq=None)

In [63]:

    
def run_farrington_model(df, train_split_ratio=0.8, alpha=0.05, years_back=1):
    """
    Trains the FarringtonFlexible model and generates predictions.

    Args:
        df (pd.DataFrame): Input data with DatetimeIndex and 'n_cases' column.
        train_split_ratio (float): Proportion of data for training.
        alpha (float): Significance level.
        years_back (int): Years for baseline reference.

    Returns:
        df_full (pd.DataFrame): Original dataframe with expected and threshold values.
        predictions (pd.DataFrame): Model predictions, including 'alarm' and 'upperbound'.
        train (pd.DataFrame): Training portion of the original data.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    if 'n_cases' not in df.columns:
        raise ValueError("Input DataFrame must contain an 'n_cases' column.")
    if not (0 < train_split_ratio < 1):
        raise ValueError("train_split_ratio must be between 0 and 1 (exclusive).")

    train_size = int(len(df) * train_split_ratio)
    if train_size == 0 or train_size == len(df):
        raise ValueError("Data size or train_split_ratio results in empty train/test set.")

    train = df.iloc[:train_size].copy()
    test = df.iloc[train_size:].copy()

    model = FarringtonFlexible(alpha=alpha, years_back=years_back)
    model.fit(train)
    predictions = model.predict(test)

    df_full = df.copy()
    df_full['threshold'] = np.nan
    df_full.loc[predictions.index, 'threshold'] = predictions['upperbound']
    df_full['expected'] = train['n_cases'].mean()

    return df_full, predictions, train

def run_farrington_model_bydatesplit(df, train_end_date, test_end_date, alpha=0.05, years_back=1):
    """
    Trains the FarringtonFlexible model and generates predictions.

    Args:
        df (pd.DataFrame): Input data with DatetimeIndex and 'n_cases' column.
        train_end_date (str): End date for the training set (YYYY-MM-DD).
        test_end_date (str): End date for the testing set (YYYY-MM-DD).
        alpha (float): Significance level.
        years_back (int): Years for baseline reference.

    Returns:
        df_full (pd.DataFrame): Original dataframe with expected and threshold values.
        predictions (pd.DataFrame): Model predictions, including 'alarm' and 'upperbound'.
        train (pd.DataFrame): Training portion of the original data.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    if 'n_cases' not in df.columns:
        raise ValueError("Input DataFrame must contain an 'n_cases' column.")
    
    # 用 train/test date 训练
    train = df.loc[:train_end_date, ['n_cases', 'n_outbreak_cases']].copy()
    test = df.loc[test_end_date:, ['n_cases', 'n_outbreak_cases']].copy()

    model = FarringtonFlexible(alpha=alpha, years_back=years_back)
    model.fit(train)
    predictions = model.predict(test)

    df_full = df.copy()
    df_full['threshold'] = np.nan
    df_full.loc[predictions.index, 'threshold'] = predictions['upperbound']
    df_full['expected'] = train['n_cases'].mean()

    return df_full, predictions, train


In [70]:
from datetime import datetime

df_full, predictions, train = run_farrington_model_bydatesplit(
                    df,
                    train_end_date="2024-12-30",
                    test_end_date="2025-01-05",
                    alpha=0.05,
                    years_back=3
                )

In [68]:
# 用 2020-2024 训练
# 用 train/test date 训练
train = df.loc[:"2024-12-30", ['n_cases', 'n_outbreak_cases']].copy()
test = df.loc["2025-01-05":, ['n_cases', 'n_outbreak_cases']].copy()

model = FarringtonFlexible(alpha=0.05, years_back=3)
model.fit(train)
predictions = model.predict(test)

In [69]:
df_full = df.copy()
df_full['threshold'] = np.nan
df_full.loc[predictions.index, 'threshold'] = predictions['upperbound']
df_full['expected'] = train['n_cases'].mean()

In [66]:
train_df = train.to_frame()
model = FarringtonFlexible()
model.fit(train_df)

c:\Users\taohe\miniconda3\envs\epysurv-dev\lib\site-packages\epysurv\models\timepoint\_base.py:47: UserWarning: The column "n_outbreak_cases" is not present in input parameter `data`. "n_cases" is treated as if it contains no outbreaks.
  'The column "n_outbreak_cases" is not present in input parameter `data`. '


FarringtonFlexible(years_back=3, window_half_width=3, reweight=True, weights_threshold=2.58, alpha=0.01, trend=True, trend_threshold=0.05, past_period_cutoff=4, min_cases_in_past_periods=5, power_transform='2/3', past_weeks_not_included=26, threshold_method='delta')

In [20]:
test_df = test.to_frame()
print(test_df)

           n_cases
start_date        
2025-01-05    4660
2025-01-12    4832
2025-01-19    4742
2025-01-26    4586
2025-02-02    4739
2025-02-09    4425
2025-02-16    4537
2025-02-23    4464
2025-03-02    4232
2025-03-09    4026
2025-03-16    3780
2025-03-23    3807
2025-03-30    3498
2025-04-06    3493
2025-04-13    3485
2025-04-20    3376
2025-04-27    3247
2025-05-04    3230
2025-05-11    3039
2025-05-18    2916
2025-05-25    2871
2025-06-01    2704
2025-06-08    2070
2025-06-15    1112


In [21]:
df['n_cases'] = df['n_cases'].astype(int)
df['n_outbreak_cases'] = df['n_cases'].apply(lambda x: 0 if x <= 4000 else x - 4000)

In [22]:
print(df)

             end_date mmwr_week  n_cases  n_outbreak_cases
start_date                                                
2019-12-29 2020-01-04         1     4111               111
2020-01-05 2020-01-11         2     4153               153
2020-01-12 2020-01-18         3     4066                66
2020-01-19 2020-01-25         4     3915                 0
2020-01-26 2020-02-01         5     3818                 0
...               ...       ...      ...               ...
2025-05-18 2025-05-24        21     2916                 0
2025-05-25 2025-05-31        22     2871                 0
2025-06-01 2025-06-07        23     2704                 0
2025-06-08 2025-06-14        24     2070                 0
2025-06-15 2025-06-21        25     1112                 0

[286 rows x 4 columns]


In [44]:
def generate_plot_from_data(df,
                            save_path,
                            train_split_ratio=0.8,
                            alpha=0.05,
                            years_back=1,
                            plot_title='FarringtonFlexible Model: Case Detection Plot',
                            xlabel='Date',
                            ylabel='Number of Cases'):
    """
    Trains a FarringtonFlexible model on the provided data, makes predictions,
    and generates a plot visualizing the results, saving it to a file.

    Args:
        df (pandas.DataFrame): DataFrame containing the time series data.
                               Must have a DateTimeIndex and a column named 'n_cases'.
        save_path (str): The full path (including filename and extension, e.g., .png)
                         where the plot image will be saved.
        train_split_ratio (float): Proportion of the data to use for training (0 to 1).
        alpha (float): Significance level for the Farrington model's threshold calculation.
        years_back (int): Number of previous years' data to consider for the baseline
                          in the Farrington model.
        plot_title (str): Title for the generated plot.
        xlabel (str): Label for the x-axis.
        ylabel (str): Label for the y-axis.

    Returns:
        None: The function saves the plot to the specified file path.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame must have a DatetimeIndex.")
    if 'n_cases' not in df.columns:
        raise ValueError("Input DataFrame must contain an 'n_cases' column.")
    if not (0 < train_split_ratio < 1):
        raise ValueError("train_split_ratio must be between 0 and 1 (exclusive).")

    df = df.sort_index()
    # Split training and testing data
    #train_size = int(len(df) * train_split_ratio)
    #if train_size == 0 or train_size == len(df):
    #    raise ValueError("Data size or train_split_ratio results in an empty train or test set.")

    #train = df.iloc[:train_size].copy()
    #test = df.iloc[train_size:].copy()
    # 用 2020-2024 训练
    train = df.loc[:'2024-12-30', ['n_cases', 'n_outbreak_cases']].copy()
    test = df.loc['2025-01-05':, ['n_cases', 'n_outbreak_cases']].copy()

    print(f"Splitting data: {len(train)} training points, {len(test)} testing points.")

    # Initialize and fit the FarringtonFlexible model



    model = FarringtonFlexible(alpha=alpha, years_back=years_back)
    print("Fitting FarringtonFlexible model...")
    model.fit(train)
    print("Model fitting complete.")

    # Predict on the test set
    print("Making predictions...")
    predictions = model.predict(test)
    print("Predictions complete.")
    # print("Prediction Columns:", predictions.columns) # Optional: for debugging

    # Prepare data for visualization
    df_full = df.copy()

    # Add threshold column - only for the test period where predictions exist
    df_full['threshold'] = np.nan # Initialize with NaN
    # Align prediction index with df_full index before assigning
    common_index = predictions.index.intersection(df_full.index)
    df_full.loc[common_index, 'threshold'] = predictions.loc[common_index, 'upperbound']

    # Approximate expected cases using the mean of the training data
    expected_value = train['n_cases'].mean()
    df_full['expected'] = expected_value # Apply to the whole series for plotting continuity

    # Create figure and axis objects
    fig, ax = plt.subplots(figsize=(12, 6), dpi=300, constrained_layout=True)

    # Plot actual cases
    ax.plot(df_full.index, df_full['n_cases'], label='Actual Cases', color='blue', marker='o', markersize=4, linestyle='-')

    # Plot expected cases
    ax.plot(df_full.index, df_full['expected'], label=f'Expected Cases (Train Mean = {expected_value:.2f})', color='green', linestyle='--')

    # Plot threshold line
    ax.plot(df_full.index, df_full['threshold'], label=f'Threshold (alpha={alpha})', color='red', linestyle='--')

    # Fill between expected and threshold
    fill_indices = df_full['threshold'].dropna().index
    if not fill_indices.empty:
        expected_for_fill = df_full.loc[fill_indices, 'expected']
        threshold_for_fill = df_full.loc[fill_indices, 'threshold']
        ax.fill_between(fill_indices, expected_for_fill, threshold_for_fill,
                    where=threshold_for_fill >= expected_for_fill,
                    color='red', alpha=0.1, label='Alert Zone')

    # Highlight alarms
    if 'alarm' in predictions.columns:
        alarm_indices = predictions[predictions['alarm']].index
        outliers = df_full.loc[alarm_indices]
        if not outliers.empty:
            ax.scatter(outliers.index, outliers['n_cases'], color='purple', label='Alarms', zorder=5, s=50)

    # Set labels, title, legend
    ax.set_title(plot_title, fontsize=14, pad=20)
    ax.set_xlabel(xlabel, fontsize=12, labelpad=10)
    ax.set_ylabel(ylabel, fontsize=12, labelpad=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    # Legend 放在图内，防止出界
    ax.legend(loc='upper left', fontsize=12)

    # Adjust layout manually
    #fig.subplots_adjust(top=0.9, bottom=0.12, left=0.1, right=0.95)

    # Save directory
    save_dir = os.path.dirname(save_path)
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")

    # Save plot
    try:
        fig.savefig(save_path, dpi=300, bbox_inches="tight", pad_inches=0.2)
        print(f"Plot saved successfully to: {save_path}")
        print("Current figure size:", fig.get_size_inches())
        # fig.show()  # optional for debug
    except Exception as e:
        print(f"Error saving plot to {save_path}: {e}")
    finally:
        plt.close(fig)


In [46]:
output_plot_path = 'farrington_cdc_plot.png'
    # For a specific path like in the original example:
    # output_plot_path = r'C:\Users\YourUser\Documents\YourFolder\farrington_simulation_plot.png'
    # Ensure the directory exists or the script has permission to create it.

    # 3. Generate and save the plot using the simulated data
try:
        generate_plot_from_data(
            df=df,
            save_path=output_plot_path,
            #train_split_ratio=0.75, # Example: changed ratio
            plot_title='FarringtonFlexible Model: CDC Pneumonia Deaths',
            xlabel='Date',
            ylabel='Number of Cases',
            alpha=0.05,
            years_back=3
        )
except Exception as e:
        print(f"\nAn error occurred during plot generation: {e}")

print("\nExample usage finished.")

Splitting data: 262 training points, 24 testing points.
Fitting FarringtonFlexible model...
Model fitting complete.
Making predictions...
Predictions complete.
Plot saved successfully to: farrington_cdc_plot.png
Current figure size: [12.  6.]

Example usage finished.
